In [ ]:
import warnings
warnings.filterwarnings('ignore')

from matplotlib import rc
import matplotlib.pyplot as plt 
rc('font',family='Malgun Gothic') # 한글
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호

import os
import pandas as pd
import seaborn as sns
import numpy as np
from IPython.core.display import HTML
import time
import re

# <font color=red>__데이터 불러오기__</font>

In [ ]:
# 파일 불러와서 변수에 저장
path = './dataset'
file_list = os.listdir(path)

data_li = []
for file in file_list:
    data = pd.read_csv(f'dataset/{file}',encoding='cp949')
    file_name = file.replace('.txt','')
    globals()[file_name] = data
    data_li.append(file_name)

In [ ]:
# purprd 구매일자 type 변경 및 년, 반기, 분기, 요일 추가
purprd['구매일자'] = pd.to_datetime(purprd['구매일자'], format='%Y%m%d')
purprd['year'] = purprd['구매일자'].dt.year
purprd['quarter'] = purprd['구매일자'].dt.quarter
purprd['weekday'] = purprd['구매일자'].dt.weekday # 월 0 ~ 일 6

def to_half(year,quarter):
    if (year==2014) & (quarter in [1,2]):
        return 1
    elif (year==2014) & (quarter in [3,4]):
        return 2
    elif (year==2015) & (quarter in [1,2]):
        return 3
    else:
        return 4
        
purprd['half'] = purprd.apply(lambda x: to_half(x['year'], x['quarter']), axis=1)

In [ ]:
# 기존고객 => 매 분기 1회 이상 구매한 고객으로 한정
# 기존고객만 남긴 dataframe 생성
all_cust = pd.pivot_table(purprd,
                         index='고객번호',
                         columns='half',
                         values='구매금액',
                         aggfunc='sum')

existing_cust_idx = all_cust.dropna().index.tolist() # 기존고객 고객번호

for data in data_li:
    try:
        globals()[data] = globals()[data].query(f'고객번호 == {existing_cust_idx}')
    except:
        pass

In [ ]:
# 연령대 묶어줌
def cat_age(age):
    if age == '19세이하':
        return 10
    elif age in ['20세~24세', '25세~29세']:
        return 20
    elif age in ['30세~34세', '35세~39세']:
        return 30
    elif age in ['40세~44세', '45세~49세']:
        return 40
    elif age in ['50세~54세', '55세~59세']:
        return 50
    else:
        return 60
    
cust['연령대'] = cust['연령대'].apply(lambda x: cat_age(x))

# <font color=red>__종속변수__</font>

In [ ]:
# 종속변수
def get_label(p1, p2):
    """
    전체 매출 증감율을 고려한 고객별 매출 증감율(반기 기준)
    -> 감소고객 : 1
    -> 증가고객 : 0
    """
    sales = pd.pivot_table(purprd,index='고객번호', # 고객별 반기 매출
                              columns = 'half',
                              values = '구매금액',
                              aggfunc= 'sum')
    rate_variation = (sum(sales[int(p2)])-sum(sales[int(p1)]))/sum(sales[int(p1)]) # 전체 매출 증감율
    sales[f'y'] = (sales[int(p2)] - sales[int(p1)])/sales[int(p1)]/rate_variation # 고객별 매출 증감율
    
    def to_label(sales_variation): # 매출 감소 고객 : 1
        if sales_variation >= 0:
            return 0
        else:
            return 1
    
    sales[f'y'] = sales[f'y'].apply(lambda x: to_label(x))
    sales = sales[[f'y']]
    return sales

# <font color=red>__독립변수__</font>

# 1) membership

In [ ]:
# membership 가입 개수
def membership_count():
    membership_cust = pd.pivot_table(membership,
                                      index='고객번호',
                                      columns='멤버십명',
                                      values='가입년월',
                                      aggfunc='count').fillna(0)
    membership_cust['가입개수'] = membership_cust.sum(axis=1)
    return membership_cust[['가입개수']]

In [ ]:
# 최초 membership 가입년도
def membership_date():
    membership['가입년월'] = pd.to_datetime(membership['가입년월'], format='%Y%m')
    membership['가입년도'] = membership['가입년월'].dt.year

    first_membership_date = pd.pivot_table(membership,
                                    index='고객번호',
                                    values='가입년도',
                                    aggfunc='min')
    return first_membership_date

# 2) channel

In [ ]:
# app login 횟수
def app_count():
    channel_count = pd.pivot_table(channel,
                  index='고객번호',
                  columns='제휴사',
                  values='이용횟수')

    channel_count['APP로그인횟수'] = channel_count[channel_count.columns[channel_count.columns.str.contains('APP')]].sum(axis=1)
    return channel_count[['APP로그인횟수']]

# 3) compuse

In [ ]:
# B제휴사 경쟁사 이용률
def B_compuse_rate():
    compuse_count = pd.pivot_table(compuse,
                    index='고객번호',
                    columns='경쟁사',
                    values='제휴사',
                    aggfunc='count').fillna(0)
    compuse_count['c_B'] = compuse_count['B01'] + compuse_count['B02']

    purprd_count = pd.pivot_table(purprd.drop_duplicates(subset='영수증번호'),
                                  index='고객번호',
                                  columns='제휴사',
                                  values='영수증번호',
                                  aggfunc='count').fillna(0)

    compuse_count = compuse_count.join(purprd_count)

    compuse_count[f'c_B_rate'] = round(compuse_count['c_B']/(compuse_count['c_B']+compuse_count['B'])*100, 2)
    return compuse_count[['c_B_rate']]

#### prodcat

In [ ]:
# 상품분류 - 대분류, 구매목적분류 추가(수작업)
cat_name = pd.read_excel('상품분류.xlsx', index_col=0)[['소분류코드','대분류','구매목적분류']]
prodcat = pd.merge(prodcat, cat_name, on=['소분류코드'])
purprd = pd.merge(purprd, prodcat[['소분류코드','중분류명','소분류명','대분류','구매목적분류']])

# 5) purprd

In [ ]:
### 비율 계산
def to_rate(df, name):
    total = df.sum(axis=1)
    for col in df.columns:
        df[f'{col}_{name}_rate'] = round(df[col]/total*100, 2)
        df.drop(col, axis=1, inplace=True)
    return df

In [ ]:
# 선매품, 편의품 구매금액 비중
def purpose_cat_amount(p1, p2):
    purpose_cat = pd.pivot_table(purprd.query(f'half==[{p1},{p2}]'),
                                      index='고객번호',
                                      columns='구매목적분류',
                                      values='구매금액',
                                      aggfunc='sum').fillna(0)
    to_rate(purpose_cat, 'amount')

    purpose_cat.drop('전문품_amount_rate',axis=1,inplace=True)
    return purpose_cat

In [ ]:
# 대분류별 구매횟수 비중
def major_cat_count(p1, p2):
    major_cat_count = pd.pivot_table(purprd.query(f'half==[{p1},{p2}]'),
                                      index='고객번호',
                                      columns='대분류',
                                      values='구매금액',
                                      aggfunc='count').fillna(0)

    to_rate(major_cat_count, 'count')
    return major_cat_count[['미용품_count_rate','스포츠레저_count_rate','패션잡화_count_rate',\
                    '의류_count_rate','인테리어_count_rate']]

In [ ]:
# 대분류별 구매금액 비중
def major_cat_amount(p1, p2):
    major_cat_amount = pd.pivot_table(purprd.query(f'half==[{p1},{p2}]'),
                                      index='고객번호',
                                      columns='대분류',
                                      values='구매금액',
                                      aggfunc='sum').fillna(0)

    to_rate(major_cat_amount, 'amount')
    return major_cat_amount[['가공식품_amount_rate','교육문화_amount_rate','기타_amount_rate',\
                     '신선식품_amount_rate','일상용품_amount_rate']]

In [ ]:
# 제휴사별 구매횟수 비중
def affiliate_count(p1, p2):
    affiliate_count = pd.pivot_table(purprd.query(f'half==[{p1},{p2}]'),
                                      index='고객번호',
                                      columns='제휴사',
                                      values='구매금액',
                                      aggfunc='count').fillna(0)
    to_rate(affiliate_count, 'count')
    return affiliate_count

In [ ]:
# 제휴사별 구매금액 비중
def affiliate_mount(p1, p2):
    affiliate_mount = pd.pivot_table(purprd.query(f'half==[{p1},{p2}]'),
                                      index='고객번호',
                                      columns='제휴사',
                                      values='구매금액',
                                      aggfunc='sum').fillna(0)
    to_rate(affiliate_mount, 'mount')
    return affiliate_mount

In [ ]:
### 증감율 계산(purprd, 구매금액 기준)
def purprd_amount_pv(col, period1, period2):
    for i in [period1, period2]:
        globals()[f'p{i}'] = pd.pivot_table(purprd.query(f'half=={i}'),
                                           index='고객번호',
                                           columns=col,
                                           values='구매금액',
                                           aggfunc='sum').fillna(0)
        
    variation = (globals()[f'p{period2}'] - globals()[f'p{period1}'])/globals()[f'p{period1}']*100
    return variation.replace({np.inf:100, np.nan:0})

# <font color=red>__dataset__</font>

In [ ]:
def make_dataset(p1, p2, p3):
    dataset = pd.DataFrame(cust[['연령대','성별']]).join([ # 연령대, 성별    -> - / label
        membership_count(), # membership 가입 개수                           
        membership_date(), # 최초 membership 가입년도                        -> label
        app_count(), # app login 횟수                                        
        B_compuse_rate(), # B제휴사 경쟁사 이용률                            
        purpose_cat_amount(p1, p2), # 선매품, 편의품 구매금액 비중           
        major_cat_count(p1, p2), # 대분류별 구매횟수 비중(미용품,스포츠레저,패션잡화,의류,인테리어) 
        major_cat_amount(p1, p2), # 대분류별 구매금액 비중(가공식품,교육문화,기타,신선식품,일상용품)
        affiliate_count(p1, p2), # 제휴사별 구매횟수 비중
        affiliate_mount(p1, p2), # 제휴사별 구매금액 비중
        purprd_amount_pv('구매목적분류', p1, p2)[['편의품']], # 편의품 구매금액 증감율
        purprd_amount_pv('대분류', p1, p2)[['가공식품','미용품','스포츠레저','의류']], # 대분류별 구매금액 증감율
        purprd_amount_pv('제휴사', p1, p2)[['A']], # 제휴사별 구매금액 증감율
        get_label(p1, p3)]) # 종속변수
    dataset.fillna(0, inplace=True)
    return dataset

dataset1 = make_dataset(1,2,3) # train(train / vaild)
dataset2 = make_dataset(2,3,4) # test

In [ ]:
# Labelencoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_cols = ['성별','가입년도']

for col in le_cols:
    dataset1[col] = le.fit_transform(dataset1[col])
    dataset2[col] = le.fit_transform(dataset2[col])

In [ ]:
# Category
# cat_cols = dataset1.columns[4:-7].tolist()

# def to_cat(df, col, n=6):
#     data = df[col].astype(float)
#     cat_data = pd.cut(data, n, labels=list(range(1, n+1)))
#     return cat_data

# for col in cat_cols:
#     dataset1[col] = to_cat(dataset1, col)
#     dataset2[col] = to_cat(dataset2, col)

In [ ]:
# # Category(증감율 => 마이너스 / 플러스 나누고 카테고리)
# cat_plus_cols = ['편의품', '가공식품', '미용품', '스포츠레저', '의류', 'A']

# def to_cat_plus(df, col, n1=3, n2=3):
#     data = df[col]
    
#     data_minus = data[data<=0]
#     data_minus_cut = pd.cut(data_minus,n1,labels=list(range(1,n1+1)))

#     data_plus = data[0<data]
#     data_plus_cut = pd.cut(data_plus,n2,labels=list(range(n1+1,n1+n2+1)))
    
#     return pd.concat([data_minus_cut,data_plus_cut])

# for col in cat_plus_cols:
#     dataset1[col] = to_cat_plus(dataset1, col)
#     dataset2[col] = to_cat_plus(dataset2, col)

In [ ]:
# Standardscaler
# from sklearn.preprocessing import MinMaxScaler

# sc = MinMaxScaler()

# for col in sc_cols:
#     dataset1[col] = sc.fit_transform(dataset1[[col]])
#     dataset2[col] = sc.fit_transform(dataset2[[col]])

In [ ]:
for col in dataset1.dtypes[dataset1.dtypes=='category'].index:
    dataset1[col] = dataset1[col].astype(int)
    dataset2[col] = dataset2[col].astype(int)

In [ ]:
dataset1.to_csv('dataset1.csv')
dataset2.to_csv('dataset2.csv')

dataset1 = pd.read_csv('dataset1.csv', index_col=0)
dataset2 = pd.read_csv('dataset2.csv', index_col=0)

In [ ]:
from sklearn.model_selection import train_test_split
X = dataset1.drop('y', axis=1)
y = dataset1['y']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.3, # vaild
                                                   random_state=1004)

In [ ]:
# #최종 TEST시 사용
# X_train = dataset1.drop('y', axis=1)
# y_train = dataset1['y']

# X_test = dataset2.drop('y', axis=1)
# y_test = dataset2['y']

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier, plot_importance
# from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


# 객체 생성
dct_clf = DecisionTreeClassifier(criterion = 'entropy')
rf_clf = RandomForestClassifier()
lr_clf = LogisticRegression()
xgb_clf = XGBClassifier()
lgb_clf = LGBMClassifier()
gb_clf = GradientBoostingClassifier()
# svm_clf = SVC(probability=True)

# 파라미터 설정
dct_parameters = {'max_depth':[3,5], }
rf_parameters = {'n_estimators':[100,200,300], 'max_depth':[7,9,11]}
lr_parameters = { "penalty":['l1', 'l2', 'elasticnet', 'none'], 'C': [ 1, 10, 100, 1000]}
xgb_parameters = {'n_estimators':[300], 'learning_rate':[0.05, 0.1], 'max_depth':[4,5,6]}
lgb_parameters = {'n_estimators':[300], 'learning_rate':[0.05, 0.1], 'max_depth':[4,5,6]}
gb_parameters = {'n_estimators':[100,200,300,400,500],'learning_rate':[0.05,0.1]}
# svm_parameters = {'kernel':['linear', 'rbf'], 'C':[2,4,6,8,10]}

clf_param = [(dct_clf,dct_parameters),(rf_clf,rf_parameters),(lr_clf,lr_parameters),
             (xgb_clf,xgb_parameters),(lgb_clf,lgb_parameters),(gb_clf,gb_parameters),
             ] # , (svm_clf,svm_parameters)

for clf, parameter in clf_param:
    grid_clf = GridSearchCV(clf, param_grid=parameter, scoring='accuracy', cv=3, refit=True)
    grid_clf.fit(X_train, y_train)
    
    # 교차검증 결과 출력
    class_name = clf.__class__.__name__
    scores_df = pd.DataFrame(grid_clf.cv_results_)
    display(HTML(scores_df[['params','rank_test_score','mean_test_score']].to_html()))
    print(f'{class_name} 최적 하이퍼 파라미터:', grid_clf.best_params_)
    print('{0} 최고 정확도:{1:.4f}'.format(class_name,grid_clf.best_score_))
    
    # x_test에 최적 하이퍼 파라미터 적용하여 분석한 결과
    best_clf = grid_clf.best_estimator_
    pred = best_clf.predict(X_test)
    pred_proba = best_clf.predict_proba(X_test)[:,1]
    print('테스트 데이터 세트 정확도:{:.4f}'.format(accuracy_score(y_test,pred)))

,params,rank_test_score,mean_test_score
0,{'max_depth': 3},2,0.692658
1,{'max_depth': 5},1,0.694299


DecisionTreeClassifier 최적 하이퍼 파라미터: {'max_depth': 5}
DecisionTreeClassifier 최고 정확도:0.6943
테스트 데이터 세트 정확도:0.7010


,params,rank_test_score,mean_test_score
0,"{'max_depth': 7, 'n_estimators': 100}",6,0.707432
1,"{'max_depth': 7, 'n_estimators': 200}",5,0.707656
2,"{'max_depth': 7, 'n_estimators': 300}",9,0.706686
3,"{'max_depth': 9, 'n_estimators': 100}",1,0.711088
4,"{'max_depth': 9, 'n_estimators': 200}",2,0.710939
5,"{'max_depth': 9, 'n_estimators': 300}",3,0.709670
6,"{'max_depth': 11, 'n_estimators': 100}",7,0.707208
7,"{'max_depth': 11, 'n_estimators': 200}",8,0.707208
8,"{'max_depth': 11, 'n_estimators': 300}",4,0.708999


RandomForestClassifier 최적 하이퍼 파라미터: {'max_depth': 9, 'n_estimators': 100}
RandomForestClassifier 최고 정확도:0.7111
테스트 데이터 세트 정확도:0.7133


,params,rank_test_score,mean_test_score
0,"{'C': 1, 'penalty': 'l1'}",9,NaN
1,"{'C': 1, 'penalty': 'l2'}",3,0.654081
2,"{'C': 1, 'penalty': 'elasticnet'}",10,NaN
3,"{'C': 1, 'penalty': 'none'}",4,0.654007
4,"{'C': 10, 'penalty': 'l1'}",11,NaN
5,"{'C': 10, 'penalty': 'l2'}",2,0.654231
6,"{'C': 10, 'penalty': 'elasticnet'}",12,NaN
7,"{'C': 10, 'penalty': 'none'}",4,0.654007
8,"{'C': 100, 'penalty': 'l1'}",13,NaN
9,"{'C': 100, 'penalty': 'l2'}",8,0.653559


...

XGBClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
XGBClassifier 최고 정확도:0.7075
테스트 데이터 세트 정확도:0.7131


,params,rank_test_score,mean_test_score
0,"{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}",1,0.706014
1,"{'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 300}",3,0.701314
2,"{'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 300}",2,0.701761
3,"{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 300}",4,0.700568
4,"{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}",6,0.694822
5,"{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 300}",5,0.696911


LGBMClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
LGBMClassifier 최고 정확도:0.7060
테스트 데이터 세트 정확도:0.7138


,params,rank_test_score,mean_test_score
0,"{'learning_rate': 0.05, 'n_estimators': 100}",1,0.708477
1,"{'learning_rate': 0.05, 'n_estimators': 200}",2,0.708253
2,"{'learning_rate': 0.05, 'n_estimators': 300}",3,0.708104
3,"{'learning_rate': 0.05, 'n_estimators': 400}",5,0.706462
4,"{'learning_rate': 0.05, 'n_estimators': 500}",6,0.705790
5,"{'learning_rate': 0.1, 'n_estimators': 100}",4,0.708029
6,"{'learning_rate': 0.1, 'n_estimators': 200}",7,0.704447
7,"{'learning_rate': 0.1, 'n_estimators': 300}",8,0.703179
8,"{'learning_rate': 0.1, 'n_estimators': 400}",9,0.700866
9,"{'learning_rate': 0.1, 'n_estimators': 500}",10,0.699896


GradientBoostingClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'n_estimators': 100}
GradientBoostingClassifier 최고 정확도:0.7085
테스트 데이터 세트 정확도:0.7117


# <font color=red>__아노바분석🤖__</font>

In [1]:
# # anova pvalue 함수👻
from scipy import stats

def anova_test(dataset):
    num = 1
    data = dataset.drop('y',axis=1)
    target = dataset.y
    data = data.join(target)
    
    for n in range(len(data.columns[:-1])):
        grps = [data[data.columns[-1]].tolist() for _, data in data.groupby(data.columns[n])]        
        F, p = stats.f_oneway(*grps)
        if p >= 0.05:
            print(num, data.columns[n],':', round(p,3),'무의미')
        elif p < 0.05:
            print(num, data.columns[n],':', round(p,3),'😊')
        num += 1
            
anova_test(dataset1)